# frontend 

In [ ]:
npx create-vite@latest

cd frontend
npm i
npm run dev

# backend

In [ ]:

uvicorn main:app --reload

python main.py

In [ ]:
optiionaly :
for using langchain -

#
groq>=0.4.1          # Compatible with langchain-groq
langchain-groq==0.1.10  # Latest version as of recent updates

#resume_analyzer.py
from groq import Groq
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

class ResumeAnalyzer:
    def __init__(self):
        # Initialize Groq client with your API key
        self.client = Groq(api_key="YOUR_GROQ_API_KEY")
        # Initialize LangChain's ChatGroq model
        self.llm = ChatGroq(
            groq_api_key="YOUR_GROQ_API_KEY",
            model_name="mixtral-8x7b-32768",
            max_tokens=1000
        )

    def analyze_resume(self, resume_text: str) -> dict:
        # Define a structured prompt using LangChain's PromptTemplate
        prompt_template = PromptTemplate(
            input_variables=["resume"],
            template="""
            You are a career coach. Analyze the following resume and provide:
            1. Personalized feedback on strengths and areas for improvement.
            2. Next steps for career growth.
            3. Specific advice to navigate their career path.
            
            Resume:
            {resume}
            
            Return the response in JSON format with keys: 'feedback', 'next_steps', 'advice'.
            Ensure the response is a valid JSON string.
            """
        )

        # Create a chain with the LLM and prompt
        chain = LLMChain(llm=self.llm, prompt=prompt_template)

        # Run the chain with the resume text
        raw_output = chain.run(resume=resume_text)
        print("Raw output from LangChain:", raw_output)  # Debug

        # Parse the response as JSON
        try:
            result = json.loads(raw_output)
            # Validate the structure
            if not all(key in result for key in ["feedback", "next_steps", "advice"]):
                raise ValueError("Missing required keys in response")
            if not isinstance(result["next_steps"], list) or not isinstance(result["advice"], list):
                raise ValueError("next_steps and advice must be lists")
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing response: {e}")
            # Fallback response
            result = {
                "feedback": "Your resume shows strong technical skills, but consider adding more quantifiable achievements.",
                "next_steps": [
                    "Update your resume with recent projects.",
                    "Pursue a certification in a trending technology."
                ],
                "advice": [
                    "Network with professionals in your field.",
                    "Explore roles in emerging industries like AI or cloud computing."
                ]
            }

        return result

